In [2]:
#coding=utf8
import torch
import torch.nn as nn
# import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import hiddenlayer as h
from visdom import Visdom
import datetime
import os
# import reuse
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
from torchviz import make_dot

# 这里很普通的检查cuda可用性
x = torch.rand(2,2)
print(x)
print(torch.cuda.is_available())

tensor([[0.3027, 0.6614],
        [0.4898, 0.1737]])
True


In [3]:
# 以下是检查点路径、监视窗口初始化
# 请在当前环境下 CMD 输入python -m visdom.server 启动监视器
# 数据处理现在已移至 emgDataprocess.ipynb
ckpDir = './/ckp//c10'
if not os.path.exists(ckpDir):
    os.makedirs(ckpDir)

timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
print(timeForSave)
vizx = 0
viz = Visdom()
viz1 = Visdom()
viz2 = Visdom()
viz3 = Visdom()
viz4 = Visdom()
viz5 = Visdom()

Setting up a new session...
Setting up a new session...


2022_01_04_22:16:23


Setting up a new session...
Setting up a new session...
Setting up a new session...
Setting up a new session...


In [2]:
# 自定义数据集类

transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])

class EMGDataset(Dataset):
 
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.transforms = transforms.ToTensor()
 
    def __getitem__(self, index):
        emgData = self.data[index,:,:,:]
        emgData = np.squeeze(emgData)#似乎不应该压缩了
        emglabel = self.label[index]
        emglabel = emglabel.astype(np.int16)
        emgData = self.transforms(emgData)      
        
        return emgData,emglabel
 
    def __len__(self):
        return len(self.label)
 
dataarray = np.load('../data/OpenganDataSet2213.npy',allow_pickle=True)
CNNdataset = dataarray.item()
print(type(CNNdataset))
traindata = CNNdataset['Xtrain']
trainlabel = CNNdataset['Ytrain']
testdata = CNNdataset['Xtest']
testlabel = CNNdataset['Ytest']
# trainunknown_data = CNNdataset['Xtrain_unknown']
# trainunknownc_label = CNNdataset['Ytrain_unknown']
# # print(trainlabel[:,0])

trainlabel = trainlabel[:,0]
testlabel = testlabel[:,0]
# trainunknownc_label = trainunknownc_label[:,0]
# print(type(trainlabel))
train_set = EMGDataset(traindata, trainlabel)
test_set = EMGDataset(testdata, testlabel)
# train_unknown = EMGDataset(trainunknown_data,trainunknownc_label)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, pin_memory=True,
#                                             num_workers=3)

sample = next(iter(test_set))
print(sample)

<class 'dict'>
(tensor([[[0.0171, 0.0073, 0.0024, 0.0024, 0.0024, 0.0024, 0.0757, 0.0317,
          0.0024, 0.0146],
         [0.0171, 0.0244, 0.0024, 0.0024, 0.0024, 0.0024, 0.0684, 0.0342,
          0.0024, 0.0146],
         [0.0146, 0.0269, 0.0024, 0.0024, 0.0024, 0.0024, 0.0610, 0.0342,
          0.0024, 0.0146],
         [0.0220, 0.0195, 0.0024, 0.0024, 0.0024, 0.0024, 0.0513, 0.0366,
          0.0024, 0.0098],
         [0.0269, 0.0171, 0.0024, 0.0024, 0.0024, 0.0024, 0.0488, 0.0415,
          0.0024, 0.0073],
         [0.0269, 0.0293, 0.0024, 0.0024, 0.0024, 0.0024, 0.0732, 0.0488,
          0.0024, 0.0073],
         [0.0244, 0.0317, 0.0024, 0.0024, 0.0024, 0.0024, 0.1001, 0.0635,
          0.0024, 0.0098],
         [0.0220, 0.0244, 0.0024, 0.0024, 0.0024, 0.0024, 0.1074, 0.0757,
          0.0024, 0.0195],
         [0.0171, 0.0146, 0.0024, 0.0024, 0.0024, 0.0024, 0.1172, 0.0854,
          0.0024, 0.0220],
         [0.0195, 0.0049, 0.0024, 0.0024, 0.0024, 0.0024, 0.1147, 0.0854,
 

In [4]:
# 自定义神经网络,Resnet
class Resnet_Basicblock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride = 1):
        super(Resnet_Basicblock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,\
            stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,\
            stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        # 保证最后一步求和时特征图的维度一致
        if stride !=1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels,\
                    kernel_size=1, stride=stride, bias = False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )


    def forward(self,x):
        # 1st 3*3, k = 64
        out = F.relu(self.bn1(self.conv1))
        # 2nd 3*3, k = 64
        out = self.bn2(self.conv2(out))
        # add input x
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    


class Resnet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(Resnet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1,\
            padding=1, bis=False)
        self.bn1 = nn.BatchNorm2d(64)


Network(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4352, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=6, bias=True)
  (dr1): Dropout2d(p=0.2, inplace=False)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [1, 32, 38, 8]             320
            Conv2d-2             [1, 32, 35, 5]           9,248
            Linear-3                   [1, 128]         557,184
         Dropout2d-4                   [1, 128]               0
            Linear-5                     [1, 6]             774
Total params: 567,526
Trainable params: 567,526
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.

'Digraph.gv.png'

In [4]:
# 自定义神经网络,CNN
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32,\
             kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32,\
             kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, \
            kernel_size=3, padding=0)

        self.fc1 = nn.Linear(in_features=32 * 18 * 8, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=10)
        self.dr1 = nn.Dropout2d(0.2)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        # t = self.dr1(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (4) hidden linear layer
        t = t.reshape(-1, 32 * 18 * 8)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.dr1(t)

        # (5) output layer
        t = self.out(t)

        return t

net = Network()
# 打印网络，检查输入输出 shape是否正确
print(net)
summary(net,(1,20,10),batch_size = 1,device = "cpu")

# 可视化结构，hiddenlayer
# vis_graph = h.build_graph(net, torch.zeros([1,1,40,10]))
# vis_graph.theme = h.graph.THEMES["blue"].copy()
# vis_graph.save("./CNNtrynetframe")
# 可视化结构，torchviz
sampleInput = torch.randn(1,1,20,10).requires_grad_(True)
sampleOutput = net(sampleInput)
print('Outputshape:',sampleOutput.shape)
framevision = make_dot(sampleOutput, params=dict(list(net.named_parameters()) + [('x',sampleInput)]))
framevision.format = "png"
framevision.direcory = "./"
framevision.view()

Network(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4608, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=10, bias=True)
  (dr1): Dropout2d(p=0.2, inplace=False)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 32, 20, 10]             320
            Conv2d-2             [1, 32, 19, 9]           9,248
            Linear-3                   [1, 128]         589,952
         Dropout2d-4                   [1, 128]               0
            Linear-5                    [1, 10]           1,290
Total params: 600,810
Trainable params: 600,810
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Fo

'Digraph.gv.png'

In [6]:
# import datetime

# timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
# print(timeForSave)
print(trainlabel.size)

4228


In [45]:
# 视情况加载已保存文件
# checkpoint_pretrain = torch.load('./ckp/c6c6_ep_1600_2021_12_05_16_38_46.pth')
# net.load_state_dict(checkpoint_pretrain['model'])
ckpDir = './/ckp//c10'
if not os.path.exists(ckpDir):
    os.makedirs(ckpDir)
timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
# 设置使用的训练设备
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
net = net.to(device)
# 加载数据，设置优化器
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000,shuffle=True)
# train_un_loader = torch.utils.data.DataLoader(train_unknown, batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100,shuffle=True)
optimizer = torch.optim.Adam(net.parameters(),
        lr=0.0002)
lr_schedule = torch.optim.lr_scheduler.StepLR(\
        optimizer, 400, gamma=0.5, last_epoch=-1)
total_test_acc = 0
total_test_correct = 0
totaltest = 0
vizx = 0
epoch_num = 2000
# 开始迭代
for epoch in range(epoch_num):

    total_loss = 0
    total_correct = 0
    curr_total_correct = 0
    i = 0
    net.train()
    # Classify real examples into the correct K classes
    total_traintnum = 0
    for batch_kn in train_loader: # Get Batch
        i+=1
        images, labels = batch_kn
        # 数据和标签转为所需数据类型
        images = images.to(torch.float32)
        labels = labels.long()
        preds = net(images.to(device)) # Pass Batch
        trainloss = F.cross_entropy(preds.to(device), labels.to(device)) # Calculate Loss
        known_augmented_logits = F.pad(preds, (0,1))
        known_log_soft_open = F.log_softmax(known_augmented_logits.to(device), dim=1)[:, -1]
        # errknown =  known_log_soft_open.mean()
        # trainloss = trainloss + errknown
        optimizer.zero_grad()
        trainloss.backward() # Calculate Gradients
        optimizer.step() # Update Weights
        total_loss += trainloss.item()
        # total_correct += get_num_correct(preds.to(device), labels.to(device))
        curr_total_correct = (reuse.get_num_correct(preds.to(device), labels.to(device)))
        total_correct += curr_total_correct
        total_traintnum += labels.size(0)
    total_train_acc = total_correct/total_traintnum
    total_correct = 0
    # Classify aux_dataset examples as open set
    # for batch_unkn in train_un_loader:
    #     un_data,_ = batch_unkn
    #     un_data = un_data.to(torch.float32)
    #     classifier_logits = net(un_data.to(device))
    #     augmented_logits = F.pad(classifier_logits, (0,1))
    #     log_soft_open = F.log_softmax(augmented_logits.to(device), dim=1)[:, -1]
    #     errOpenSet = -log_soft_open.mean()
    #     optimizer.zero_grad()
    #     errOpenSet.backward()
    #     optimizer.step() # Update Weights

    # 可视化
    vizx+=1
    viz.line([float(trainloss)],[vizx],\
        win='trainloss', update='append',opts=dict(title='trainloss'))   
    viz2.line([float(optimizer.state_dict()['param_groups'][0]['lr'])],[vizx],\
        win='lr', update='append',opts=dict(title='lr'))
    viz3.line([float(total_train_acc)],[vizx],\
        win='train_acc', update='append',opts=dict(title='train_acc'))
    # viz5.line([float(errOpenSet)],[vizx],\
    #     win='errOpenSet', update='append',opts=dict(title='erropen'))   
    # 测试
    net.eval()
    total_testnum = 0
    for testemgdatas, testemglabels in test_loader: # Get Batch
        testemgdatas = testemgdatas.to(torch.float32)
        testemglabels = testemglabels.long()
        predstest = net(testemgdatas.to(device))
        testloss = F.cross_entropy(predstest.to(device), testemglabels.to(device)) # Calculate Loss
        curr_test_correct = (get_num_correct(predstest.to(device), testemglabels.to(device)))
        total_testnum += testemglabels.size(0)
        total_test_correct += curr_test_correct
        # totaltest += testemglabels.size(0)
    # total_test_acc = total_test_correct/(trainlabel.size)
    total_test_acc = total_test_correct/total_testnum
    
    viz1.line([float(testloss)],[vizx],win='testloss', update='append',opts=dict(title='testloss'))
    viz4.line([float(total_test_acc)],[vizx],\
        win='test_acc', update='append',opts=dict(title='test_acc'))
    total_test_correct = 0

    print(
        "epoch", epoch, 
        "total_train_acc:", total_train_acc, 
        "loss:", total_loss,
        "test_loss:",float(testloss),
        "test_acc:",total_test_acc
    )
   # 更新学习率
    lr_schedule.step()
   # 定期保存
    if epoch%50 == 1:
        timeForSave = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        checkpointPath = ckpDir+'//c10_ep_'+str(epoch)+'_'+timeForSave+'.pth'
        state = {'model': net.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
        torch.save(state, checkpointPath)
FinalPath = 'c10s'+timeForSave+'.pth'
torch.save(net.state_dict(),FinalPath)


epoch 0 total_train_acc: 0.17478713339640492 loss: 11.292065620422363 test_loss: 2.235746145248413 test_acc: 0.20526960784313725
epoch 1 total_train_acc: 0.21665089877010407 loss: 10.881728410720825 test_loss: 2.0872159004211426 test_acc: 0.2261029411764706
epoch 2 total_train_acc: 0.25733207190160834 loss: 10.523176431655884 test_loss: 2.1652181148529053 test_acc: 0.27144607843137253
epoch 3 total_train_acc: 0.29730368968779564 loss: 10.059801936149597 test_loss: 1.9547343254089355 test_acc: 0.3002450980392157
epoch 4 total_train_acc: 0.3098391674550615 loss: 9.6390141248703 test_loss: 1.7625138759613037 test_acc: 0.31924019607843135
epoch 5 total_train_acc: 0.3403500473036897 loss: 9.344666361808777 test_loss: 1.7929034233093262 test_acc: 0.3321078431372549
epoch 6 total_train_acc: 0.36258278145695366 loss: 8.936329245567322 test_loss: 1.689946174621582 test_acc: 0.39950980392156865
epoch 7 total_train_acc: 0.39120151371807 loss: 8.66004490852356 test_loss: 1.6948480606079102 test_ac

In [4]:
checkpoint_for_vector = torch.load('./c10s2022_01_03_20_17_36.pth')
net_vector = Network()
net_vector.load_state_dict(checkpoint_for_vector)
net_vector.eval()
emg_vector_num = 0
vector_Dataloader = torch.utils.data.DataLoader(train_set, batch_size=1,shuffle=False)

In [48]:
# 计算数据集的网络预测输出并构建新的特征向量四维 array
# 注意迭代器只能建立一次，然后以此前进，所以这里单列一行，避免重复建设
# 达成类似目标的做法有很多种，例如 append() 等，这里先完成功能，优化待后来同学了
def emgdata_to_net_preds(data_set,net_vector):
    batchl = iter(data_set)
    emg_vec = [torch.tensor([],requires_grad=False) for i in range(len(data_set.label))]
    for idx, _ in enumerate(emg_vec):
        sample_data,sample_label = next(batchl)
        sample_data = sample_data.to(torch.float32).unsqueeze(0)
        sample_label = torch.as_tensor(sample_label).long()
        emg_vec[idx] = net_vector(sample_data).detach().numpy()
    emg_vec_np = np.array(emg_vec)
    emg_vec_np = emg_vec_np[:,np.newaxis,:,:]
    return emg_vec_np

In [49]:
emg_vec_npp = emgdata_to_net_preds(data_set=train_set, net_vector = net_vector)
# np.save('../data/OpenganDatafetrevec220111.npy',emg_vec)


In [ ]:
# batch = [batch for batch in train_set]
batch = next(batchl)
# print(batch)
sample_data,sample_label = batch
# sample_data,sample_label = next(iter(vector_Dataloader))
sample_data = sample_data.to(torch.float32)
sample_data = sample_data.unsqueeze(0)
sample_label = torch.as_tensor(sample_label)
sample_label = sample_label.long()
# print(sample_data)
pre_vector = net_vector(sample_data)
print(pre_vector)
print(pre_vector.shape)
# print(batch)


In [153]:
emg_vec_batch = pre_vector
print(emg_vec_batch.shape)

sample_datas= next(iter(train_set))
sample_data,sample_label = sample_datas
sample_data = sample_data.to(torch.float32)
sample_data = sample_data.unsqueeze(0)
sample_label = torch.as_tensor(sample_label)
sample_label = sample_label.long()
print(sample_data)
pre_vector = net_vector(sample_data)
print(pre_vector)

torch.Size([1, 10])
tensor([[[[0.0269, 0.0293, 0.0024, 0.0024, 0.0024, 0.0024, 0.0732, 0.0488,
           0.0024, 0.0073],
          [0.0244, 0.0317, 0.0024, 0.0024, 0.0024, 0.0024, 0.1001, 0.0635,
           0.0024, 0.0098],
          [0.0220, 0.0244, 0.0024, 0.0024, 0.0024, 0.0024, 0.1074, 0.0757,
           0.0024, 0.0195],
          [0.0171, 0.0146, 0.0024, 0.0024, 0.0024, 0.0024, 0.1172, 0.0854,
           0.0024, 0.0220],
          [0.0195, 0.0049, 0.0024, 0.0024, 0.0024, 0.0024, 0.1147, 0.0854,
           0.0024, 0.0220],
          [0.0269, 0.0049, 0.0024, 0.0024, 0.0024, 0.0024, 0.1147, 0.0806,
           0.0024, 0.0146],
          [0.0342, 0.0171, 0.0024, 0.0024, 0.0024, 0.0024, 0.1221, 0.0781,
           0.0024, 0.0098],
          [0.0391, 0.0171, 0.0024, 0.0024, 0.0024, 0.0024, 0.1147, 0.0732,
           0.0024, 0.0049],
          [0.0488, 0.0146, 0.0024, 0.0024, 0.0024, 0.0024, 0.1001, 0.0659,
           0.0024, 0.0024],
          [0.0464, 0.0122, 0.0024, 0.0024, 0.0049, 0.

In [ ]:
# 使用训练好的模型，用其输出构建新数据集
# 这段感觉应该放在数据处理的，但好像不是那么好切割开，就先这样吧
checkpoint_for_vector = torch.load('./c10s2022_01_03_20_17_36.pth')
net_vector = Network()
net_vector.load_state_dict(checkpoint_for_vector)
net_vector.eval()
# emg_vector = []
# emg_vector = np.array(emg_vector)
emg_vector_num = 0
vector_Dataloader = torch.utils.data.DataLoader(train_set, batch_size=1,shuffle=False)
for emgdata, emglabel in vector_Dataloader:
    emgdata = emgdata.to(torch.float32)
    emglabel = emglabel.long()
    pre_vector = net_vector(emgdata)
    emg_vector[emg_vector_num,0,:,:] = pre_vector
    emg_vector_num += 1
print(emg_vector.shape)


In [15]:
# 重新加载模型
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)

        self.fc1 = nn.Linear(in_features=32 * 4 * 34, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=6)
        self.dr1 = nn.Dropout2d(0.2)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        # t = self.dr1(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (4) hidden linear layer
        t = t.reshape(-1, 32 * 4 * 34)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.dr1(t)

        # (5) output layer
        t = self.out(t)

        return t
        
net_eval = Network()

checkpoint_eval = torch.load('./ckp/c6addnewloss/gap5_c6op_ep_600_2021_12_16_15_47_32.pth')
net_eval.load_state_dict(checkpoint_eval['model'])
# net_eval.load_state_dict(checkpoint_eval)

<All keys matched successfully>

In [7]:
# 自定义数据集类

transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])

class EMGDataset(Dataset):
 
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.transforms = transform
 
    def __getitem__(self, index):
        emgData = self.data[index,:,:,:]
        emgData = np.squeeze(emgData)#似乎不应该压缩了
        emglabel = self.label[index]
        emglabel = emglabel.astype(np.int16)
        emgData = self.transforms(emgData)      
        
        return emgData,emglabel
 
    def __len__(self):
        return len(self.label)
 
 
# if __name__ == '__main__':
dataarray = np.load('../data/OpenSetDataSet.npy',allow_pickle=True)
CNNdataset = dataarray.item()
print(type(CNNdataset))
opensetdata = CNNdataset['X_oo']
opensetlabel = CNNdataset['Y_oo']
# trainlabel = CNNdataset['Ytrain']
# testdata = CNNdataset['Xtest']
# testlabel = CNNdataset['Ytest']
# # print(trainlabel[:,0])

opensetlabel = opensetlabel[:,0]
# testlabel = testlabel[:,0]
# print(type(trainlabel))
open_set = EMGDataset(opensetdata, opensetlabel)

# train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, pin_memory=True,
#                                             num_workers=3)
openset_loader = torch.utils.data.DataLoader(open_set, batch_size=1,shuffle=True)
sample = next(iter(openset_loader))
print(sample)

<class 'dict'>
[tensor([[[[0.1587, 0.0293, 0.1221, 0.0439, 0.0024, 0.0024, 0.3979, 0.4346,
           0.2100, 0.1123],
          [0.1587, 0.0293, 0.1270, 0.0439, 0.0049, 0.0049, 0.3979, 0.4590,
           0.2710, 0.1196],
          [0.1636, 0.0317, 0.1270, 0.0439, 0.0049, 0.0049, 0.3882, 0.4810,
           0.3247, 0.1221],
          [0.1563, 0.0342, 0.1172, 0.0391, 0.0024, 0.0024, 0.3931, 0.4639,
           0.3564, 0.1123],
          [0.1514, 0.0317, 0.1025, 0.0342, 0.0024, 0.0024, 0.4053, 0.4517,
           0.3491, 0.1074],
          [0.1392, 0.0220, 0.0806, 0.0244, 0.0024, 0.0024, 0.3882, 0.4297,
           0.3223, 0.0977],
          [0.1196, 0.0098, 0.0610, 0.0171, 0.0024, 0.0024, 0.3491, 0.3906,
           0.3101, 0.0854],
          [0.1025, 0.0024, 0.0439, 0.0073, 0.0024, 0.0024, 0.3003, 0.3516,
           0.2905, 0.0757],
          [0.1050, 0.0024, 0.0366, 0.0024, 0.0024, 0.0024, 0.2686, 0.3784,
           0.2588, 0.0732],
          [0.1123, 0.0024, 0.0342, 0.0024, 0.0024, 0.0024

In [16]:
# 效果评估
openset_loader = torch.utils.data.DataLoader(open_set, batch_size=1,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1,shuffle=True)
openset_scores = []
openset_scores2 = []
# 可视化
viz6 = Visdom()
viz7 = Visdom()
vizx = 0
openscoresave = []
for opensetdata, opensetlabel in openset_loader: # Get Batch
        opensetdata = opensetdata.to(torch.float32)
        opensetlabel = opensetlabel.long()
        logits = net_eval(opensetdata)
        augmented_logits = F.pad(logits, pad=(0,1))
        preds = F.softmax(augmented_logits)
        prob_unknown = preds[:, -1]
        prob_known = preds[:, :-1].max(dim=1)[0]
        prob_open = prob_unknown - prob_known
        openset_scores.extend(prob_open.data.cpu().numpy())
        prob_openforvis = prob_open.float()
        viz6.line([float(prob_openforvis)],[vizx],\
                win='openscore', update='append',opts=dict(title='openscore for unknown'))
        vizx += 1

vizx = 0
for testemgdatas, testemglabels in test_loader: # Get Batch
        testemgdatas = testemgdatas.to(torch.float32)
        testemglabels = testemglabels.long()
        logits2 = net_eval(testemgdatas)
        augmented_logits2 = F.pad(logits2, pad=(0,1))
        preds2 = F.softmax(augmented_logits2)
        prob_unknown2 = preds2[:, -1]
        prob_known2 = preds2[:, :-1].max(dim=1)[0]
        prob_open2 = prob_unknown2 - prob_known2
        openset_scores2.extend(prob_open2.data.cpu().numpy())
        prob_openforvis2 = prob_open2.float()
        viz7.line([float(prob_openforvis2)],[vizx],\
                win='openscore2', update='append',opts=dict(title='openscore for known'))
        vizx += 1
# for i, (images, labels) in enumerate(dataloader):
#     images = Variable(images, volatile=True)
#     logits = netC(images)
#     augmented_logits = F.pad(logits, pad=(0,1))
#     # The implicit K+1th class (the open set class) is computed
#     #  by assuming an extra linear output with constant value 0
#     preds = F.softmax(augmented_logits)
#     #preds = augmented_logits
#     prob_unknown = preds[:, -1]
#     prob_known = preds[:, :-1].max(dim=1)[0]
#     prob_open = prob_unknown - prob_known
#     openset_scores.extend(prob_open.data.cpu().numpy())

Setting up a new session...
Setting up a new session...
C:\Users\cwdbo\AppData\Local\Temp/ipykernel_9732/2954200349.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = F.softmax(augmented_logits)
C:\Users\cwdbo\AppData\Local\Temp/ipykernel_9732/2954200349.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds2 = F.softmax(augmented_logits2)
